In [77]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
# #from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [78]:
# Reading the data
rheas_tamsat_raw=pd.read_csv('data/rheas_tamsat/ken_n_25_tamsat_cultivar1_fert5_2015_2023_maize.csv')
# rheas_tamsat_raw=pd.read_csv('data/rheas_tamsat/maize_ken_n_25_tamsat.csv')

In [79]:
# change dates to months or years
rheas_tamsat_raw['planting']=pd.to_datetime(rheas_tamsat_raw['planting'], format='ISO8601')
rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='ISO8601')
rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='ISO8601')
rheas_tamsat_raw['year'] = rheas_tamsat_raw['planting'].dt.year
rheas_tamsat_raw['year'] = rheas_tamsat_raw['planting'].dt.year
# rheas_tamsat_raw['harvest']=pd.to_datetime(rheas_tamsat_raw['harvest'], format='%Y/%m/%d')


# calculate yield
rheas_tamsat_raw['yield']=rheas_tamsat_raw['gwad']/1000


In [80]:
# unimodal season
unimodal_season =[9, 10, 11, 12, 1,2, 3, 4]
unimodal_season = ['NONE', 'NONE', 'MAM', 'MAM','MAM', 'NONE', 'NONE', 'NONE', 'NONE', 'OND', 'OND', 'OND']
month_to_season=dict(zip(range(1,13), seasons))
month_to_season
rheas_tamsat_raw['season']= rheas_tamsat_raw['planting'].dt.month.map(month_to_season)

# bimodal season
bimodal_season = [1, 2, 3, 4]
bimodal_season = ['NONE', 'NONE', 'MAM', 'MAM','MAM', 'NONE', 'NONE', 'NONE', 'NONE', 'OND', 'OND', 'OND']

# # Extract data only for MAM and OND
rheas_tamsat_MAM= rheas_tamsat_raw.loc[rheas_tamsat_raw['season']=='MAM']
rheas_tamsat_OND=rheas_tamsat_raw.loc[rheas_tamsat_raw['season']=='OND']
# extract data for 2023 only
rheas_tamsat_MAM_2023=rheas_tamsat_MAM[rheas_tamsat_MAM['year']==2023]
rheas_tamsat_OND_2022=rheas_tamsat_MAM[rheas_tamsat_MAM['year']==2022]

# aggregate 
rheas_tamsat_MAM_2023=rheas_tamsat_MAM.groupby(['cname'], as_index=False).agg({'yield': 'mean'})
rheas_tamsat_OND_2022=rheas_tamsat_OND.groupby(['cname'], as_index=False).agg({'yield': 'mean'})

In [1]:
# Read shapefiles and rename columns
country_shapefile = gpd.read_file("shp/gadm36_TZA_2_LongRain.shp")
tanzania_unimodal_shp = gpd.read_file("shp/gadm36_TZA_2_LongRain.shp")
tanzania_bimodal_shp = gpd.read_file("shp/gadm36_TZA_2_Bimodal.shp")

# Rename kenya_shp 
tanzania_unimodal_shp=tanzania_unimodal_shp.rename(columns={'NAME1':'cname'})
tanzania_bimodal_shp=tanzania_unimodal_shp.rename(columns={'NAME1':'cname'})

NameError: name 'gpd' is not defined

In [ ]:
# Merge data into shapefiles
unimodal = pd.merge(tanzania_unimodal_shp, rheas_tamsat_unimodal_2022, on="cname")
bimodal = pd.merge(tanzania_bimodal_shp, rheas_tamsat_bimodal_2022, on="cname")

In [82]:
geo_source_unimodal = GeoJSONDataSource(geojson=unimodal.to_json())
geo_source_bimodal = GeoJSONDataSource(geojson=bimodal.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [90]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p1 = figure(title='Long Rains  Season MAM Yield (Mt/Ha) - 2023',
           outer_width=650, outer_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_unimodal,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('Province: ', '@cname'), ('Yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [84]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
p2 = figure(title='Long Rains Season MAM Yield (Mt/Ha) - 2022',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source_bimodal,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('Province: ', '@cname'), ('Yield', '@yield')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [85]:
# Short Rains Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p3 = figure(title='Short Rains Season OND Yield(Mt/Ha) - 2022',
#            outer_width=650, outer_height=600)

# p3.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p3.patches('xs','ys', source=geo_source_OND,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
# p3.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p3.add_layout(color_bar, 'right')

In [86]:
# Short Rains Season 2022
# color_mapper1 = LinearColorMapper(palette="RdBu10", low=3.0, high=3.5)
# p4 = figure(title='Short Rains Season OND Yield(Mt/Ha) - 2021',
#            outer_width=650, outer_height=600)

# p4.multi_line('xs','ys', source=country_geo_source,
#             line_color='black', line_width=0.5)

# p4.patches('xs','ys', source=geo_source_OND,
#           fill_color={'field': 'yield', 'transform': color_mapper1},
#           fill_alpha=0.7, line_color='black', line_width=0.5)

# hover = HoverTool(tooltips=[('County', '@cname'), ('Yield', '@yield')])
# p4.add_tools(hover)

# color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
#                      border_line_color=None, location=(0,0), orientation='vertical')
# p4.add_layout(color_bar, 'right')

In [87]:

# p=gridplot([[p1, p2], [p3, p4]], width=650, height=600)
show(p1)

In [88]:
# from bokeh.io import save
# # specify the output file name and location
# save(p, filename='yield_map.html', title='Yield Data Map')

In [89]:
# from bokeh.io import save

# specify the output file name and location
# save(p, filename='yield_map.html', title='Yield Data Map')